In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [2]:
#colunas que escolhi para dados escolares
col_escol = ['CODMEC','ANO','TIPOESC','DEP','Qtde_Docentes','NUM_FUNC','MERENDA','CLFUND',
            'ALFUND','TOTALEJA','TOT_EE','TOTAL_ALUNOS','CL_CRECHE','AL_CRECHE','CL_PRE','AL_PRE','CLEM','ALEM']
col_rendi = ['CODMEC','EFABAND9ANOS8S','EFABAND9ANOS9S','EFABAND7S','EFABAND8S','TOTALEF_ABAND','TOTALEF_APROV',
             'TOTALEF_REPROV','TOTAL_EM_ABAND','TOTAL_EM_REPROV','CONCLUINTES_EM','EPABAND','EPREPROV']
dados_escolares = pd.read_csv('DadosEscolares1996-2015.csv', encoding = 'latin1', usecols = col_escol)
rendimento = pd.read_csv('RendimentoEscolar2000-2015.csv', encoding = 'latin1', usecols = col_rendi)
esc_rmsp = pd.read_csv('ESC2013_RMSP_CEM.csv', encoding = 'latin1')
nota = pd.read_csv('ENEM2015.csv')

C:\Users\giova\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#verificando o tamanho
print ( 'Dados escolares' + str(dados_escolares.shape))
print ( 'Rendimento' + str(rendimento.shape))
print ( 'ESC_RMSP' + str(esc_rmsp.shape))
print( 'Nota' + str(nota.shape))

Dados escolares(104292, 18)
Rendimento(14944, 13)
ESC_RMSP(10937, 136)
Nota(1243, 3)


In [4]:
#Existem escolas que estão no dataframe dados_escolares, rendimento, esc_rmsp que eu não tenho o valor da nota dela
#Preciso retirar escolas que não estão no dataframe nota
dados_escolares = dados_escolares[dados_escolares.CODMEC.isin(nota['CODIGO DA ENTIDADE'])]
rendimento = rendimento[rendimento.CODMEC.isin(nota['CODIGO DA ENTIDADE'])]
esc_rmsp = esc_rmsp[esc_rmsp.CODESC.isin(nota['CODIGO DA ENTIDADE'])]

In [8]:
#retirando as colunas em que mais que 50% dos valores são null
dados_escolares1 = dados_escolares.drop(dados_escolares.columns[dados_escolares.isnull().mean()>0.5], axis = 1)
#apagando as linha com valor NaN
dados_escolares1.dropna(inplace = True, how = 'any')

In [9]:
#como eu abri todas as colunas do esc_rmsp, agora vou retirar as que não desejo usar
col_rmsp = ['ID', 'LONGITUDE', 'LATITUDE', 'CODESCTX','NOMEESC', 'NOMEMUN','CODMUN', 'NOMDIST', 'CODDIST', 'COD_SC',
           'BAIRRO','CEP','END_ESC','NUM_ESC','DDD','TELEFONE','ENS_REG','REG_IN_CR','REG_IN_PE','REG_FU_8A',
           'REG_FU_9A','REG_MED','REG_MED_I','REG_MED_N','REG_MED_P','ENS_ESP','ESP_IN_CR','ESP_IN_PE',
           'ESP_FU_8A','ESP_FU_9A','ESP_MED','ESP_MED_I','ESP_MED_N','ESP_MED_P','ESP_EJA_F','ESP_EJA_M','EJA',
           'EJA_FUND','EJA_MED','EJA_PROJ']
esc_rmsp1 = esc_rmsp.drop(col_rmsp, axis = 1)

In [10]:
esc_rmsp1.drop(esc_rmsp1.columns[esc_rmsp1.isnull().mean() > 0.3], axis=1, inplace = True)
esc_rmsp1.dropna(how='any', axis = 0, inplace = True)

In [11]:
print('Dados escolares', dados_escolares1.isnull().sum().sum())
print('Rendimento', rendimento.isnull().sum().sum())
print('ESC_RMSP1', esc_rmsp1.isnull().sum().sum())

Dados escolares 0
Rendimento 0
ESC_RMSP1 0


In [12]:
#primeiro teste vai ser o mais simples e fácil, usar o maior dataset que tenho
teste1 = pd.merge(dados_escolares1, nota, how = 'inner', left_on = 'CODMEC', right_on = 'CODIGO DA ENTIDADE')
teste1.drop(['ANO', 'TIPOESC', 'CODIGO DA ENTIDADE', 'NUMERO DE PARTICIPANTES'], axis = 1, inplace = True)
dep = {'Privada': 'privada', 'Estadual':'estadual', 'PRIVADA':'privada', 'ESTADUAL':'estadual',
      'privada':'privada', 'PARTICULAR':'privada','Municipal':'municipal','MUNICIPAL':'municipal',
      'Federal':'federal', 'FEDERAL':'federal'}
teste1.DEP = teste1.DEP.map(dep)
teste1 = pd.get_dummies(teste1, 'DEP')

In [13]:
target = teste1.Classificacao
data = teste1.drop('Classificacao', axis =1)

In [14]:
data.CODMEC  = data.CODMEC.astype('float64')
data_TOTALEJA = data.TOTALEJA.astype('float64')
data.TOTAL_ALUNOS = data.TOTAL_ALUNOS.astype('float64')
data.DEP_estadual = data.DEP_estadual.astype('float64')
data.DEP_federal = data.DEP_federal.astype('float64')
data.DEP_municipal = data.DEP_municipal.astype('float64')
data.DEP_privada = data.DEP_privada.astype("float64")
data.dtypes

CODMEC           float64
CL_CRECHE        float64
CL_PRE           float64
AL_CRECHE        float64
AL_PRE           float64
CLEM             float64
ALEM             float64
CLFUND           float64
ALFUND           float64
TOTALEJA         float64
TOTAL_ALUNOS     float64
DEP_estadual     float64
DEP_federal      float64
DEP_municipal    float64
DEP_privada      float64
dtype: object

In [15]:
target.value_counts()

2    7249
3    1355
1    1130
4     656
0       3
Name: Classificacao, dtype: int64

In [16]:
def modelo_svm_kf(data, target, gamma, C, k):
    kf = KFold(n_splits = k)
    scores = []
    for g in gamma:
            for c in C:
                mean_kf = []
                for train_i, test_i in kf.split(data):
                    #separação dos dados em treino e teste
                    data_train, data_test = data.iloc[train_i], data.iloc[test_i]
                    target_train, target_test = target[train_i], target[test_i]
                    
                    #normalização dos dados
                    scaler = StandardScaler()
                    scaler.fit(data_train)
                    data_train = scaler.transform(data_train)
                    data_test = scaler.transform(data_test)
                    
                    #pca nos dados
                    pca = PCA(0.95)
                    pca.fit(data_train)
                    data_train = pca.transform(data_train)
                    data_test = pca.transform(data_test)
                    
                    #rodando o svm
                    svc = SVC(kernel = 'rbf', C=c, gamma = g)
                    svc.fit(data_train,target_train)
                    mean_kf.append(svc.score(data_test,target_test))
                                   
                #salvando a média do KFold para g e c dados
                scores.append(((sum(mean_kf)/len(mean_kf)), g, c))
                print('Iteração com gamma %f e c %f concluída' %(g, c))
    return scores

In [17]:
modelo_svm_kf(data, target, [15], [15], 5)

Iteração com gamma 15.000000 e c 15.000000 concluída


[(0.6400440076089741, 15, 15)]

In [18]:
gamma = np.linspace(0.1,20,15)
c = gamma
svm1 = modelo_svm_kf(data, target, gamma, c,5)

Iteração com gamma 0.100000 e c 0.100000 concluída
Iteração com gamma 0.100000 e c 1.521429 concluída
Iteração com gamma 0.100000 e c 2.942857 concluída
Iteração com gamma 0.100000 e c 4.364286 concluída
Iteração com gamma 0.100000 e c 5.785714 concluída
Iteração com gamma 0.100000 e c 7.207143 concluída
Iteração com gamma 0.100000 e c 8.628571 concluída
Iteração com gamma 0.100000 e c 10.050000 concluída
Iteração com gamma 0.100000 e c 11.471429 concluída
Iteração com gamma 0.100000 e c 12.892857 concluída


KeyboardInterrupt: 

In [24]:
svm1.sort()
svm1[-5:]

[(0.6972936663023285, 11.47142857142857, 0.1),
 (0.6973898201039683, 14.314285714285711, 0.1),
 (0.6973898663985285, 10.049999999999999, 0.1),
 (0.6974859276110479, 7.2071428571428555, 0.1),
 (0.6985438971964476, 0.1, 0.1)]

In [19]:
from sklearn.model_selection import train_test_split
data_train, data_test, target_train, target_test = train_test_split(data, target, test_size = 0.2)
svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
svr_rbf.fit(data_train, target_train)
print(svr_rbf.score(data_train, target_train))
svr_rbf.score(data_test, target_test)

0.9789630066886671


-0.004511907131504289

In [44]:
new_df = teste1[teste1.Classificacao != 0].groupby('Classificacao')
n_df = new_df.apply(lambda x : x.sample(n = 500))
n_df

CODMEC  CL_CRECHE  CL_PRE  AL_CRECHE  AL_PRE  CLEM  \
Classificacao                                                               
1             6874   35004297        0.0     0.0        0.0     0.0  16.0   
              9855   35207639        0.0     0.0        0.0     0.0  10.0   
              5555   35001168        0.0     0.0        0.0     0.0  18.0   
              5610   35001284        0.0     0.0        0.0     0.0  18.0   
              6202   35002707        0.0     0.0        0.0     0.0  23.0   
              6091   35002461        0.0     0.0        0.0     0.0  14.0   
              7349   35036274        0.0     0.0        0.0     0.0  11.0   
              5842   35001855        0.0     0.0        0.0     0.0  21.0   
              6216   35002707        0.0     0.0        0.0     0.0  23.0   
              6149   35002562        0.0     0.0        0.0     0.0  36.0   
              6320   35003001        0.0     0.0        0.0     0.0  16.0   
              7296   35005435        0.0     0.0        0.0     0.0  18.0   
              6886   35004297        0.0     0.0        0.0     0.0  13.0   
              6994   35004625        0.0     0.0        0.0     0.0  24.0   
              6381   35003384        0.0     0.0        0.0     0.0  20.0   
              6405   35003402        0.0     0.0        0.0     0.0  40.0   
              5906   35001971        0.0     0.0        0.0     0.0  37.0   
              7305   35005435        0.0     0.0        0.0     0.0  18.0   
              5841   35001855        0.0     0.0        0.0     0.0  37.0   
              4194   35153564        0.0     8.0        0.0    61.0   3.0   
              6526   35003657        0.0     0.0        0.0     0.0  11.0   
              7518   35044441        0.0     0.0        0.0     0.0  22.0   
              6168   35002636        0.0     0.0        0.0     0.0  31.0   
              10264  35924398        0.0     0.0        0.0     0.0   7.0   
              10047  35812912        6.0     4.0       50.0    29.0   0.0   
              6354   35003244        0.0     0.0        0.0     0.0  22.0   
              5840   35001855        0.0     0.0        0.0     0.0  54.0   
              5571   35001247        0.0     0.0        0.0     0.0  15.0   
              7469   35038295        0.0     0.0        0.0     0.0  21.0   
              7140   35005061        0.0     0.0        0.0     0.0   9.0   
...                       ...        ...     ...        ...     ...   ...   
4             1659   35105314        2.0     6.0       24.0    86.0   8.0   
              8046   35104620        0.0     0.0        0.0     0.0  18.0   
              2193   35107402        0.0     8.0        0.0   153.0  20.0   
              9705   35165347        0.0     0.0        0.0     0.0   9.0   
              3209   35137133        3.0     6.0       35.0    83.0   3.0   
              2300   35107542        4.0    11.0       50.0   199.0   7.0   
              8003   35104449        0.0     0.0        0.0     0.0   9.0   
              8610   35126718        0.0     0.0        0.0     0.0   9.0   
              9569   35158483        0.0     0.0        0.0     0.0  13.0   
              4493   35162243       14.0    15.0      175.0   291.0  22.0   
              486    35101278       11.0     7.0       98.0    90.0  17.0   
              3203   35137133        4.0     5.0       48.0    87.0   3.0   
              2784   35127991        7.0     9.0       74.0   126.0   0.0   
              8133   35106636        0.0     0.0        0.0     0.0   6.0   
              1672   35105314        0.0     3.0        0.0    69.0   9.0   
              2142   35107190        1.0     6.0       15.0   119.0   8.0   
              8904   35134806        0.0     1.0        0.0     8.0  33.0   
              9257   35141240        0.0     0.0        0.0     0.0   7.0   
              8023   35104607        0.0     2.0        0.0    32.0  10.0   
              8137 

In [181]:
from random import randint
target = n_df.Classificacao
data = n_df.drop(['CODMEC','Classificacao'], axis = 1)
s = StandardScaler()
data = s.fit_transform(data)
data_train, data_test, target_train, target_test = train_test_split(data, target, test_size = 0.2, random_state = randint(1,15))
svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
svr_rbf.fit(data_train, target_train)
print(svr_rbf.score(data_train, target_train))
svr_rbf.score(data_test, target_test)

C:\Users\giova\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\giova\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


0.7903147965725161


0.5977329219256182

In [57]:
target.value_counts()

4    500
3    500
2    500
1    500
Name: Classificacao, dtype: int64

In [90]:
from sklearn.model_selection import GridSearchCV
params = {'C':[100],'gamma':[0.1], 'epsilon':[0.1]}
svr = SVR()
clf = GridSearchCV(svr, params, cv = 5)
clf.fit(data, target)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [100], 'gamma': [0.1], 'epsilon': [0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [87]:
sorted(clf.cv_results_.keys())

['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'mean_train_score',
 'param_C',
 'param_gamma',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split0_train_score',
 'split1_test_score',
 'split1_train_score',
 'split2_test_score',
 'split2_train_score',
 'split3_test_score',
 'split3_train_score',
 'split4_test_score',
 'split4_train_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score',
 'std_train_score']

In [93]:
clf.cv_results_['mean_train_score']

array([0.76447464])

In [91]:
clf.cv_results_['mean_test_score']

array([-1.40096483])

In [72]:
from sklearn.linear_model import LinearRegression

In [73]:
lm = LinearRegression()

In [175]:
data_train, data_test, target_train, target_test = train_test_split(data, target, test_size = 0.2)
lm.fit(data_train, target_train)
print(lm.score(data_train, target_train))
lm.score(data_test, target_test)

0.5887376566289555


0.5665067041485381

In [78]:
lm.coef_

array([ 0.0311115 , -0.20090988,  0.02465854,  0.17099259,  0.13493284,
       -0.60069225, -0.24630744, -0.48599317, -0.0880357 ,  1.22104533,
       -0.44493945, -0.02291303, -0.10251283,  0.4654865 ])

In [79]:
data.shape

(2000, 14)

In [80]:
n_df.columns

Index(['CODMEC', 'CL_CRECHE', 'CL_PRE', 'AL_CRECHE', 'AL_PRE', 'CLEM', 'ALEM',
       'CLFUND', 'ALFUND', 'TOTALEJA', 'TOTAL_ALUNOS', 'Classificacao',
       'DEP_estadual', 'DEP_federal', 'DEP_municipal', 'DEP_privada'],
      dtype='object')

In [109]:
from sklearn.model_selection import cross_val_score
clf1 = SVR(kernel='rbf', C=100, gamma= 0.1)
scores = cross_val_score(clf1, data, target, cv=5)

In [171]:
target = target.droplevel(level = 0)

In [173]:
s = StandardScaler()
data = s.fit_transform(data)
kf = KFold(n_splits = 5)
for train_i, test_i in kf.split(data):
    data_train, data_test = data[train_i], data[test_i]
    target_train, taget_test = target.iloc[train_i], target.iloc[test_i]
    svr = LinearRegression()
    svr.fit(data_train, target_train)
    print(svr.score(data_train, target_train))
    print(svr.score(data_test, target_test))

0.5823053822593998
-0.5178970369169149
0.6041526891768623
-0.5675656320958788
0.5796769492594731
-0.5598815115876852
0.5844190587452349
-0.627106608047707
0.5803955320854002
-0.6246720818293618


In [180]:
target.iloc[[0,1]]

4488    4
5535    1
Name: Classificacao, dtype: int64

In [153]:
n_df = n_df.sample(frac = 1)

In [155]:
data = n_df.drop(['CODMEC', 'Classificacao'], axis = 1)
target = n_df.Classificacao

In [159]:
n_df

CODMEC  CL_CRECHE  CL_PRE  AL_CRECHE  AL_PRE  CLEM  \
Classificacao                                                               
4             4488   35162243        0.0    16.0        0.0   480.0   5.0   
1             5535   35001036        0.0     0.0        0.0     0.0  24.0   
3             2833   35131660        1.0    12.0       21.0   251.0  20.0   
              245    35100473       10.0    13.0      122.0   222.0  14.0   
              8708   35132214        0.0     0.0        0.0     0.0  10.0   
1             6021   35002203        0.0     0.0        0.0     0.0  30.0   
4             8114   35104985        0.0     0.0        0.0     0.0  38.0   
3             749    35101886        2.0     4.0        8.0    71.0   5.0   
1             5759   35001685        0.0     0.0        0.0     0.0   7.0   
3             1176   35102805        0.0    15.0        0.0   263.0  12.0   
1             5270   35000413        0.0     0.0        0.0     0.0  17.0   
3             5977   35002185        0.0     0.0        0.0     0.0  22.0   
4             3199   35137133        8.0     7.0      111.0   144.0   5.0   
2             9148   35139099        0.0     0.0        0.0     0.0   3.0   
1             6404   35003402        0.0     0.0        0.0     0.0  42.0   
3             2917   35132986        4.0     2.0       45.0    27.0   3.0   
2             4784   35173666        0.0     0.0        0.0     0.0   2.0   
1             5532   35001036        0.0     0.0        0.0     0.0  19.0   
              7349   35036274        0.0     0.0        0.0     0.0  11.0   
4             9247   35141240        0.0     0.0        0.0     0.0   6.0   
3             5282   35000486        0.0     0.0        0.0     0.0   9.0   
1             7466   35038295        0.0     0.0        0.0     0.0  33.0   
2             4112   35152523        3.0     2.0       26.0    32.0   5.0   
4             3744   35142736        0.0     5.0        0.0    83.0   3.0   
              8135   35106636        0.0     0.0        0.0     0.0   6.0   
              2189   35107402        0.0    14.0        0.0   205.0  31.0   
2             5165   35811385        0.0     3.0        0.0    31.0   6.0   
4             1722   35105648        0.0    19.0        0.0   550.0  37.0   
3             4020   35151506        4.0     7.0       32.0   105.0   7.0   
4             8137   35106636        0.0     1.0        0.0    31.0  19.0   
...                       ...        ...     ...        ...     ...   ...   
              2634   35124199        2.0     6.0       30.0   129.0  13.0   
2             96     35100213        3.0     4.0        9.0    23.0   9.0   
3             1521   35104991        0.0    11.0        0.0   147.0  10.0   
1             6761   35004121        0.0     0.0        0.0     0.0  23.0   
4             3479   35140338        8.0     9.0       92.0   169.0   6.0   
3             8545   35124175        0.0     0.0        0.0     0.0   4.0   
              10277  35990012        0.0     0.0        0.0     0.0  76.0   
1             7330   35035622        0.0     0.0        0.0     0.0   0.0   
4             2132   35107190        1.0     4.0       16.0    83.0   9.0   
3             8054   35104875        0.0    11.0        0.0   113.0   9.0   
2             3188   35137108        2.0     6.0       29.0    75.0   5.0   
              4576   35165098        7.0    10.0       14.0    53.0   5.0   
3             8587   35125726        0.0     4.0        0.0    68.0   0.0   
              1449   35103861        3.0     7.0       37.0   127.0  17.0   
4             8104   35104985        0.0     0.0        0.0     0.0  42.0   
2             6062   35002422        0.0     0.0        0.0     0.0  54.0   
              5501   35000954        0.0     0.0        0.0     0.0  39.0   
3             4602   35165323        6.0     6.0       38.0    64.0   3.0   
              790    35101904        5.0     8.0       61.0   164.0  11.0   
1             7390 

In [170]:
target.droplevel(level=0)

4488     4
5535     1
2833     3
245      3
8708     3
6021     1
8114     4
749      3
5759     1
1176     3
5270     1
5977     3
3199     4
9148     2
6404     1
2917     3
4784     2
5532     1
7349     1
9247     4
5282     3
7466     1
4112     2
3744     4
8135     4
2189     4
5165     2
1722     4
4020     3
8137     4
        ..
2634     4
96       2
1521     3
6761     1
3479     4
8545     3
10277    3
7330     1
2132     4
8054     3
3188     2
4576     2
8587     3
1449     3
8104     4
6062     2
5501     2
4602     3
790      3
7390     1
8696     3
5895     1
8595     4
10192    4
2296     4
253      2
1511     3
5255     1
7460     1
6858     1
Name: Classificacao, Length: 2000, dtype: int64